## Character based context as token based fails in Finnish

In [1]:
# from chatgpt:

def old_find_repeated_substrings(sentence):
    def get_substrings(s, min_len=3):
        substrings = {}
        for start in range(len(s)):
            for end in range(start + min_len, len(s) + 1):
                substr = s[start:end]
                if substr in substrings:
                    substrings[substr].append((start, end - 1))
                else:
                    substrings[substr] = [(start, end - 1)]
        return substrings

    all_substrings = get_substrings(sentence)
    repeated_substrings = {k: v for k, v in all_substrings.items() if len(v) > 1}
    return repeated_substrings

# Example usage:
sentence = "I have a cat. The catto's name is Mia"
result = old_find_repeated_substrings(sentence)
print(result)


{' ca': [(8, 10), (17, 19)], ' cat': [(8, 11), (17, 20)], 'cat': [(9, 11), (18, 20)]}


Does not fully work; asking for modification:

In [22]:
import re

def find_repeated_substrings(sentence):
    def get_substrings(s, min_len=4):
        substrings = {}
        for match in re.finditer(r'\b\w+', s):
            start = match.start()
            word = match.group()
            for i in range(len(word) - min_len + 1):
                for j in range(i + min_len, len(word) + 1):
                    substr = word[i:j]
                    actual_start = start + i
                    actual_end = start + j - 1
                    if substr in substrings:
                        substrings[substr].append((actual_start, actual_end))
                    else:
                        substrings[substr] = [(actual_start, actual_end)]
        return substrings
    
    all_substrings = get_substrings(sentence)
    
    # Filter substrings to only those that appear more than once
    # this bit written without chatgpt
    repeated_substrings = {k: v for k, v in all_substrings.items() if len(v) > 1 and all([v_ not in range(v[1][0], v[1][1]) for v_ in v[0]])}

    maximal_substrings = {}
    for substr, positions in repeated_substrings.items():
        if not any(substr in other and len(substr) < len(other) for other in repeated_substrings):
            maximal_substrings[substr] = positions
    
    return maximal_substrings

# Example usage:
#sentence = "I have a cat. The catto's name is Mia"
#result = find_repeated_substrings(sentence)
#print(result)


In [26]:
sentence = "Sairaanhoitajanne nimi on Amanda. Amandalle voi soittaa numeroon 0442003333."
result = find_repeated_substrings(sentence)
print(result)
print(sentence[26:31+1])
print(sentence[34:39+1])
print(sentence[81:84+1])
print(sentence[90:93+1])

{'Amanda': [(26, 31), (34, 39)]}
Amanda
Amanda




In [27]:
fake_tokens = ["Sairaanhoitajanne", "nimi", "on", "Ama", "#nda","#.", "Aman", "#dalle", "voi", "soittaa", "nume", "#roon", "044", "#2003333", "#."]


In [33]:
continuation_marker="#"
def which_to_mask(char_ids, tokens):
    char_ranges = [range(*i) for v in char_ids.values() for i in v]
    print(char_ranges)
    #print(char_ranges)
    indices = []
    current=0
    now=0
    #for word, char_i in char_ids.items():
    for i,t in enumerate(tokens):
        print("-------------------")
        print(i,t)
        print("now: ", now, "sentence[now]", sentence[now])
        if any([now in v for v in char_ranges]):
            print(f'adding for word {t}')
            indices.append(i)
        if t[0] == continuation_marker:
            now += len(t)-len(continuation_marker)-1
        else:
            now += len(t)+1
    return indices

print(which_to_mask(find_repeated_substrings(sentence), fake_tokens))
        
        
        

[range(26, 31), range(34, 39)]
-------------------
0 Sairaanhoitajanne
now:  0 sentence[now] S
-------------------
1 nimi
now:  18 sentence[now] n
-------------------
2 on
now:  23 sentence[now] o
-------------------
3 Ama
now:  26 sentence[now] A
adding for word Ama
-------------------
4 #nda
now:  30 sentence[now] d
adding for word #nda
-------------------
5 #.
now:  32 sentence[now] .
-------------------
6 Aman
now:  32 sentence[now] .
-------------------
7 #dalle
now:  37 sentence[now] n
adding for word #dalle
-------------------
8 voi
now:  41 sentence[now] l
-------------------
9 soittaa
now:  45 sentence[now] o
-------------------
10 nume
now:  53 sentence[now] a
-------------------
11 #roon
now:  58 sentence[now] m
-------------------
12 044
now:  61 sentence[now] o
-------------------
13 #2003333
now:  65 sentence[now] 0
-------------------
14 #.
now:  71 sentence[now] 3
[3, 4, 7]


## Okay, problems. Moving onto aligning function from other project:

In [34]:

def find_repeated_substrings(sentence):
    def get_substrings(s, min_len=4):
        substrings = {}
        for match in re.finditer(r'\b\w+', s):
            start = match.start()
            word = match.group()
            for i in range(len(word) - min_len + 1):
                for j in range(i + min_len, len(word) + 1):
                    substr = word[i:j]
                    actual_start = start + i
                    actual_end = start + j - 1
                    if substr in substrings:
                        substrings[substr].append((actual_start, actual_end))
                    else:
                        substrings[substr] = [(actual_start, actual_end)]
        return substrings
    
    all_substrings = get_substrings(sentence)
    
    # Filter substrings to only those that appear more than once
    # this bit written without chatgpt; removing only one found and those that overlap (e.g. 3333 => 333 and 333)
    repeated_substrings = {k: v for k, v in all_substrings.items() if len(v) > 1 and all([v_ not in range(v[1][0], v[1][1]) for v_ in v[0]])}

    maximal_substrings = {}
    for substr, positions in repeated_substrings.items():
        if not any(substr in other and len(substr) < len(other) for other in repeated_substrings):
            maximal_substrings[substr] = positions
    
    return maximal_substrings

def make_score_mask(sentence, substrings):
    # Tokenize the original sentence
    tokens = re.findall(r"\w+|[^\w\s]", sentence)

    # Initialize vector with 0s
    vector = [0] * len(tokens)

    # Mark tokens that are part of maximal substrings
    for substr, positions in substrings.items():
        for start, end in positions:
            substr_token_indices = [i for i, token in enumerate(tokens) if sentence.find(token, start) == start]
            for index in substr_token_indices:
                vector[index] = 1

    return tokens, vector

def align(parsed, tokenized, scores):
    """
    Function to align two different tokenizations of the same text.
    E.g. for Chinese, the model tokenizer might tokenize
    母语 (mother tongue) as "母" and "语".
    -> if you have defined a better tokenized, give both tokenizations
    to this function and it will align them and the scores
    associated with the model tokenizer.
    All punctuation is removed for this tokenization.
    -> would be removed in the keyword extraction step anyway.
    """

    parsed = [re.sub(r'[^\D\s]|[^\w\s]', '', p) for p in parsed if any(j.isalpha() for j in p)]
    to_be_dropped = [i for i in range(len(tokenized)) if any(j.isalpha() for j in tokenized[i]) and tokenized[i] not in special_tokens]
    tokenized = [re.sub(r'[^\D\s]|[^\w\s]', '',tokenized[i]) for i in to_be_dropped]
    scores = [scores[i] for i in to_be_dropped]


    assert "".join(parsed)=="".join(tokenized), ASSERTION_MSG(parsed, tokenized, special_tokens)

    # align
    # t_ind contains the current index we're at now in the tokenized (by model) sentence
    t_ind=0
    agg_scores = np.zeros(len(parsed))
    # for each "real tokenization" (here "parsed")
    for p_ind in range(len(parsed)):
        sub_scores = []
        for p in parsed[p_ind]:     # for each char in that tokenization
            if p == tokenized[t_ind][0]:        # if theres a match
                sub_scores.append(scores[t_ind])    # add the score for the character
                if len(tokenized[t_ind])>1:         # remove the character for next calculation
                    tokenized[t_ind] = tokenized[t_ind][1:]
                else:
                    tokenized[t_ind]="-"        # not necessary but helped debug
                    t_ind+=1                    # move to next index
            else:
                raise Exception("Alignment impossible for unforeseen reasons.")       # despite assertion, something went wrong
        agg_scores[p_ind] = np.mean(sub_scores)     # aggregate the scores TODO: method

    return parsed, agg_scores.tolist()





#align(parsed_text, tokenized_text, scores)

In [39]:
sentence = "Sairaanhoitajanne nimi on Amanda. Amandalle voi soittaa numeroon 0442003333. ab c defg abcdefg"
result = find_repeated_substrings(sentence)
print(result)
tok_sentence, mask = make_score_mask(sentence, result)
for t, m in zip(tok_sentence, mask):
    print(t,m)

{'Amanda': [(26, 31), (34, 39)], 'defg': [(82, 85), (90, 93)]}
Sairaanhoitajanne 0
nimi 0
on 0
Amanda 1
. 0
Amandalle 1
voi 0
soittaa 0
numeroon 0
0442003333 0
. 0
ab 0
c 0
defg 1
abcdefg 0
